In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,7)

In [13]:
dtrain = pd.read_csv('data/merchants_train.csv', encoding='cp1251', delimiter=';')

In [14]:
dtest = pd.read_csv('data/merchants_test.csv', encoding='cp1251', delimiter=';')

In [15]:
tract = pd.read_csv('data/transactions.csv', encoding='cp1251', delimiter=',')

In [6]:
tract.describe()

,merchant_id,latitude,longitude
count,2.493530e+05,249353.000000,249353.000000
mean,4.441047e+05,49.761256,33.758102
std,2.506589e+05,17.722934,17.774541
min,1.780000e+02,-83.445345,-176.155650
25%,3.639860e+05,55.604737,30.382891
50%,4.102950e+05,55.750347,37.594847
75%,5.879220e+05,55.855153,37.661401
max,1.934268e+06,84.830761,171.866882


In [7]:
tract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249353 entries, 0 to 249352
Data columns (total 5 columns):
merchant_id              249353 non-null int64
latitude                 249353 non-null float64
longitude                249353 non-null float64
real_transaction_dttm    249353 non-null object
record_date              249353 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 9.5+ MB


In [74]:
dtrain[dtrain.longitude < 5]

,merchant_id,latitude,longitude


In [37]:
tract = tract[(tract.latitude > 10) & (tract.longitude > 10)]

In [43]:
dtrain.head()

,merchant_id,latitude,longitude
0,178,55.055553,82.911833
1,267,55.726877,37.579645
2,357,54.706402,20.471420
3,505,64.553811,40.551964
4,532,55.768355,49.217176


In [49]:
dtest.head()

,merchant_id,latitude,longitude
0,361,NaN,NaN
1,428,NaN,NaN
2,490,NaN,NaN
3,1175,NaN,NaN
4,1239,NaN,NaN


In [44]:
tract.head()

,merchant_id,latitude,longitude,real_transaction_dttm,record_date
1,178,55.055995,82.912991,17:49:50,17:54:24
3,178,55.056034,82.912734,17:49:50,17:43:01
4,178,55.056034,82.912734,17:49:50,17:45:17
7,178,55.055995,82.912991,17:49:50,17:51:34
8,267,55.741642,37.502628,15:34:28,15:32:51


In [ ]:
def str_to_time(a):
    retur

In [27]:
np.array('12:34:45'.split(':')).astype(int).dot(np.array([24 * 60, 60, 1]))

19365

In [29]:
f = lambda x: np.array(x.split(':')).astype(int).dot(np.array([24 * 60, 60, 1]))

In [38]:
np.corrcoef(np.vstack((np.abs(tract.record_date.apply(f).values - tract.real_transaction_dttm.apply(f).values),
             np.abs(tract.real_latitude.values - tract.latitude.values))))

array([[ 1.        , -0.00556875],
       [-0.00556875,  1.        ]])

In [33]:
tract.record_date.apply(f).values

array([14796, 27744, 14842, ..., 13816, 13897, 13977])

In [32]:
?np.corrcoef

In [64]:
lat_mean = tract.groupby(tract.merchant_id).latitude.mean()[dtrain.merchant_id.values].values
lon_mean = tract.groupby(tract.merchant_id).longitude.mean()[dtrain.merchant_id.values].values

In [65]:
lat_med = tract.groupby(tract.merchant_id).latitude.median()[dtrain.merchant_id.values].values
lon_med = tract.groupby(tract.merchant_id).longitude.median()[dtrain.merchant_id.values].values

In [66]:
np.mean(np.abs(lat_mean - dtrain.latitude.values) < 0.002)

0.041499537179882751

In [67]:
np.mean(np.abs(lon_mean - dtrain.longitude.values) < 0.002)

0.022832459117556311

In [69]:
np.mean(np.abs(lat_med - dtrain.latitude.values) < 0.002)

0.21320580067880285

In [70]:
np.mean(np.abs(lon_med - dtrain.longitude.values) < 0.002)

0.17078062326442456

In [82]:
dtest['_ID_'] = dtest.merchant_id
lat_med = tract.groupby(tract.merchant_id).latitude.median()[dtest.merchant_id.values].values
lon_med = tract.groupby(tract.merchant_id).longitude.median()[dtest.merchant_id.values].values
dtest['_LAT_'] = lat_med
dtest['_LON_'] = lon_med
dtest._LAT_.fillna(0, inplace=True)
dtest._LON_.fillna(0, inplace=True)
del dtest['merchant_id']
del dtest['latitude']
del dtest['longitude']

In [84]:
dtest.to_csv('task2_outputs/output_median.csv', index=False)

In [10]:
a = tract.groupby(tract.merchant_id)

In [ ]:
a['latitude'].apply()

In [ ]:
def a(x):
    sx = np.sort(x.values)
    r = np.cumsum(np.diff(sx))
    

In [12]:
a = np.array([1, 2 ,3, 4, 4])
sa = np.sort(a)
np.diff(a)

array([1, 1, 1, 0])

In [9]:
?a.apply

In [18]:
tract.groupby(tract.merchant_id).latitude.apply()

AttributeError: Cannot access attribute 'values' of 'SeriesGroupBy' objects, try using the 'apply' method

In [19]:
tract['real_latitude'] = 0
for ID, lat in dtrain[['merchant_id', 'latitude']].values:
    tract.loc[tract.merchant_id == ID, 'real_latitude'] = lat

In [21]:
?np.corrcoef